***Author**: Alexander Telle, DASU - Transferzentrum für Digitalisierung, Analytics & Data Science Ulm*

***Data Source**: [Kaggle](https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification)*

***Code Repository**:*

***Licence**:*

***Optuna Documentation**: [https://optuna.readthedocs.io/en/stable/](https://optuna.readthedocs.io/en/stable/)*

# [III. Innovationskongress Data Science](https://studium.hs-ulm.de/de/research/Seiten/Innokongress.aspx)
#### Workshop: Hyperparameter-Tuning (HPO) mit Optuna
---

> **Import der Bibliotheken & allgemeine Einstellungen**:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import CosineAnnealingLR

import torch
import torch.nn as nn

In [ ]:
EPOCHS = 50
CLASSES = 3

torch.manual_seed(42)
random.seed(42)
RANDOM_SEED = 42

> **Definition der Hyperparameter (Auswahl)**:

In [ ]:
NN_ARCHITECTURE = [15, 10]
BATCH_SIZE = 100
OPTIMIZER = 'Adam' # SGD vs. Adam vs. RMSprop
LEARNING_RATE = 1
WEIGHT_DECAY = 1
SCHEDULER = True

> **Vorbereitung der Daten**:

In [ ]:
data = pd.read_csv('Data.csv')
data.head()

In [ ]:
len(data)

In [ ]:
data.isnull().any().any()

In [ ]:
data.fetal_health.value_counts().plot(kind='bar')
data.fetal_health.value_counts()

In [ ]:
null_accuracy = data.fetal_health.value_counts()[1.0]/len(data)
null_accuracy

In [ ]:
encoder = OrdinalEncoder()
oe_columns = ['fetal_health']
encoder.fit(data[oe_columns])
data[oe_columns] = encoder.transform(data[oe_columns])

In [ ]:
data.fetal_health.value_counts()

> **Erstellung von Helfer-Klassen & -Methoden**:

In [ ]:
class FetalHealthData(torch.utils.data.Dataset):
    def __init__(self, data):
        self.labels = data.fetal_health.tolist()
        self.features = data.drop(columns=['fetal_health'], axis=1).values.tolist()
    
    def __getitem__(self, index):
        sample = np.array(self.features[index]), np.array(self.labels[index])
        return sample
        
    def __len__(self):
        return len(self.labels)

In [ ]:
# HP: Hiden Layer 0, Hidden Layer 1
def get_model():
    
    ###
    layer_0 = NN_ARCHITECTURE[0]
    layer_1 = NN_ARCHITECTURE[1]
    ###
    
    layers = list()
    
    # 21 Input Features
    in_features = len(data.drop(columns=['fetal_health'], axis=1).columns)
    
    # Input Layer -> Hidden Layer 0
    layers.append(nn.Linear(in_features, layer_0))
    layers.append(nn.LeakyReLU())
    
    # Hidden Layer 0 -> Hidden Layer 1
    layers.append(nn.Linear(layer_0, layer_1))
    layers.append(nn.LeakyReLU())
    
    # Hidden Layer 1 -> Output Layer (3 Classes)
    layers.append(nn.Dropout())
    layers.append(nn.Linear(layer_1, CLASSES))

    return nn.Sequential(*layers)

In [ ]:
# HP: Batch Size
def get_data():
    
    ###
    batch_size = BATCH_SIZE
    ###
    
    training_data, testing_data = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED, stratify=data.fetal_health)
    training_data, testing_data = FetalHealthData(training_data), FetalHealthData(testing_data)
    return torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True), torch.utils.data.DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [ ]:
# HP: Optimizer, Learning Rate, Weight Decay
def get_optimizer(model):
    
    ###
    optimizer = OPTIMIZER
    learning_rate = LEARNING_RATE
    weight_decay = WEIGHT_DECAY
    ###
    
    if optimizer == 'Adam':
        return torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=WEIGHT_DECAY)
    elif optimizer == 'SGD':
        return torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=WEIGHT_DECAY)
    elif optimizer == 'RMSprop':
        return torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=WEIGHT_DECAY)

> **Erstellung des Trainings-Loops**:

In [ ]:
# HP: Scheduler
def train(model, training_batches, testing_batches):
    ###
    scheduler = SCHEDULER
    ###
    
    accuracy = list()
    criterion = nn.CrossEntropyLoss()
    optimizer = get_optimizer(model)
    
    if scheduler:
        scheduler = CosineAnnealingLR(optimizer, EPOCHS-1, verbose=False)
    
    for epoch in range(EPOCHS):
        ### Training
        model.train()
        for samples, labels in training_batches:
            optimizer.zero_grad()
            outputs = model(samples.float())
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
        
        num_samples = 0
        correct_predictions = 0
        ### Testing
        model.eval()
        with torch.no_grad():
            for samples, labels in testing_batches:
                output = model(samples.float())
                correct_predictions += (output.argmax(dim=1) == labels).sum().item()
                num_samples += labels.size(0)
            
        accuracy.append(100.0 * correct_predictions / num_samples)
    
    return accuracy

> **Training & Evaluation**:

In [ ]:
model = get_model()
training_batches, testing_batches = get_data()
history = train(model, training_batches, testing_batches)

In [ ]:
plt.plot(history)
plt.ylabel('validation accuracy')
plt.xlabel('epoch')
plt.grid()

In [ ]:
history[-1]